# 合表

## 注意

1. `rpy2`来调用R代码，需要设`R_HOME`，否则会崩溃。
2. `%load_ext rpy2.ipython`之后，可以有`%%R`开头的纯R单元格，可以用`-i`传数据给R，或者`-o`提取数据，详情问ChatGPT。

In [26]:
import pandas as pd
import os
import sys
import numpy as np
sys.path.insert(0, '..')  # 将上一层目录添加到系统路径


from src.utils import *

reg_df = pd.concat([pd.read_csv('data/' + x) for x in os.listdir('data/')]).reset_index(drop=True)
reg_df['child_enroll'] = reg_df['child_enroll'].apply(lambda x:np.nan if x >1 else x)

In [27]:
desc(reg_df,drop=['pid','_wav','eth','enroll2','child_birth_prov'])

,count,min,max,mean,mode
wav,10302,2014.0,2020.0,2016.88,2014.0
child_birth_y,7705,1999.0,2010.0,2004.65,2004.0
child_age,10302,10.0,16.0,12.43,10.0
child_gender,10302,0.0,1.0,0.54,1.0
child_enroll,10182,0.0,1.0,0.96,1.0
child_math,10022,1.0,4.0,2.33,3.0
child_eduy,10236,0.0,13.0,6.00,6.0
mother_birth_y,6289,1938.0,1994.0,1977.47,1976.0
mother_age,9187,21.0,76.0,39.26,37.0
mother_gender,9190,0.0,1.0,0.00,0.0


In [28]:
## 合并出生省份

import os
import re
import pandas as pd
import numpy as np
import seaborn as sns
import sys

sys.path.insert(0, '..')  # 将上一层目录添加到系统路径

from src.reader import read_cfps, find_files, find_cfps

data_path = os.getenv('data_path')



In [29]:
find_files(data_path + '\\cfps',r'.*adult.*\.sas')

['C:\\Users\\lee\\nutcloud\\Data\\cfps\\2010\\cfps2010adult_202008.sas7bdat',
 'C:\\Users\\lee\\nutcloud\\Data\\cfps\\2010\\cfps2010adult_202008.sas7bdat.feather']

In [30]:
df = read_cfps('C:\\Users\\lee\\nutcloud\\Data\\cfps\\2010\\cfps2010adult_202008.sas7bdat')
df.head()

C:\Users\lee\nutcloud\Data\cfps\2010\cfps2010adult_202008.sas7bdat
feather存在，读取feather。


,pid,fid,cid,provcd,countyid,PSU,Rswt_Nat,Rswt_Res,subpopulation,subsample,...,foccupcode,foccupisco,FPARTY,MBIRTHY,moccupcode,moccupisco,MPARTY,Interviewerid,ReleaseVersion,wav
0,110001101.0,110001.0,11830.0,11.0,45.0,45.0,41928.792969,31068.037109,6.0,1.0,...,NaN,NaN,NaN,1938.0,NaN,NaN,NaN,11518.0,5.2,2010
1,110001102.0,110001.0,11830.0,11.0,45.0,45.0,37480.898438,25093.058594,6.0,1.0,...,NaN,NaN,NaN,1934.0,NaN,NaN,NaN,11518.0,5.2,2010
2,110003101.0,110003.0,11830.0,11.0,45.0,45.0,43325.535156,32052.169922,6.0,1.0,...,20603.0,2411.0,4.0,1943.0,80000.0,NaN,4.0,11518.0,5.2,2010
3,110003102.0,110003.0,11830.0,11.0,45.0,45.0,38129.796875,27176.064453,6.0,1.0,...,20904.0,2331.0,1.0,1948.0,80000.0,NaN,4.0,11518.0,5.2,2010
4,110005101.0,110005.0,11830.0,11.0,45.0,45.0,69329.976562,46488.992188,6.0,1.0,...,40301.0,5122.0,4.0,1965.0,NaN,NaN,4.0,11518.0,5.2,2010


In [31]:
birth_prov = df[['pid','QA102ACODE']]
birth_prov.columns = ['mother_pid','mother_birth_prov']

birth_prov.loc[birth_prov['mother_birth_prov'] < 0,'mother_birth_prov'] = np.nan
birth_prov.head()

C:\Users\lee\AppData\Local\Temp\ipykernel_1636\3316188949.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  birth_prov.loc[birth_prov['mother_birth_prov'] < 0,'mother_birth_prov'] = np.nan


,mother_pid,mother_birth_prov
0,110001101.0,21.0
1,110001102.0,21.0
2,110003101.0,32.0
3,110003102.0,61.0
4,110005101.0,11.0


In [32]:
from src.utils import safe_merge

reg_df = safe_merge(reg_df.drop('mother_birth_prov',axis=1),birth_prov)

Merge的方式是how='left'
注意：合并后的 DataFrame 有 5741/10302 个重复键值。第一个重复键值为: mother_pid    140394105.0
Name: 163, dtype: float64


In [33]:
desc(reg_df,drop=['pid','_wav','eth','enroll2','child_birth_prov'])

,count,min,max,mean,mode
wav,10302,2014.0,2020.0,2016.88,2014.0
child_birth_y,7705,1999.0,2010.0,2004.65,2004.0
child_age,10302,10.0,16.0,12.43,10.0
child_gender,10302,0.0,1.0,0.54,1.0
child_enroll,10182,0.0,1.0,0.96,1.0
child_math,10022,1.0,4.0,2.33,3.0
child_eduy,10236,0.0,13.0,6.00,6.0
mother_birth_y,6289,1938.0,1994.0,1977.47,1976.0
mother_age,9187,21.0,76.0,39.26,37.0
mother_gender,9190,0.0,1.0,0.00,0.0


## 回归


In [34]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [35]:
%%R -i reg_df
ls()

[1] "reg_df"


$$
Y_{it} = \beta_0 + \beta_1 S^{M}_{ipkt} + \beta_2 X_{it} + \theta^{M}_k + \tau^{M}_p + \omega_t + u_{it}
$$

In [36]:
%%R
library(fixest)
source('../src/screenreg_unicode.R')

lm0 <- feols(child_math ~ mother_eduy, data = reg_df)
lm1 <- feols(child_math ~ mother_eduy + child_age + child_gender | wav + mother_birth_y + mother_birth_prov, data = reg_df)


check(list(lm0,lm1))
# 数学成绩1是优，4是差，因此母亲的教育对子女的数学成绩有显著的正影响


                                Model 1      Model 2    
--------------------------------------------------------
(Intercept)                        2.66 ***             
                                  (0.02)                
mother_eduy                       -0.05 ***    -0.05 ***
                                  (0.00)       (0.00)   
child_age                                       0.10 ** 
                                               (0.01)   
child_gender                                    0.02    
                                               (0.01)   
--------------------------------------------------------
Num. obs.                       8294         5017       
R^2 (full model)                   0.04         0.10    
R^2 (proj model)                                0.06    
Adj. R^2 (full model)              0.04         0.08    
Adj. R^2 (proj model)                           0.06    
Num. groups: wav                                3       
Num. groups: mother_birth_y   

NOTE: 2,008 observations removed because of NA values (LHS: 280, RHS: 1,784).
NOTE: 5,285 observations removed because of NA values (LHS: 280, RHS: 1,784, Fixed-effects: 5,020).


## 内生性问题

OLS的回归可能有偏，因为误差项可能包含未被观察到的**环境**和**基因**等因素，可能同时影响母亲的schoolling和子女的outcome。

把义务教育政策的实施作为母亲教育程度的工具变量，关键的假定是CSLs只通过母亲的教育程度影响子女的outcome。

顾虑：

1. 省级层面的CSLs的实施本身也不是随机的，这可能和各省的经济发展水平等有关。在公式1中，省份固定效应处理了不随时间变化的因素，队列固定效应处理了一致变化但未被观察的因素。
    - 有些因素是随着时间变化，影响全部人，比如经济的增长，改革开放等等。那么不同出生年份的人之间的差异，可能来自这些未被观察到的共同因素。控制了队列固定效应，等于在同一个年代出生的人之间比较，那么随着年代变化的共同因素，经济增长等，就不再影响估计（被控制住了）
        - 一致变化：某些因素同时影响全国人，比如改革开放。控制了队列效应，那么等于控制了这些一致变化因素。但是有些因素可能各省份不同，比如同一年，不同省份出台了不同的政策。即使是同一年出生的人，她们的差异可能来自这些不一致的变化。
    - 两个样本的差异，可能来自她们出生在不同的省份。控制了省份固定效应，等于在同一个省份内比较，那么样本之间因为省份差异而产生的不同，就被控制住了。
        - 不随时间变化：控制了省份固定效应，那么等于同一个省份内比较。但是同一个省份的人，依然可能出生在不同等年代，那么2个样本之间的差异依然可能来同一个省份的不同时代的差异（省的GDP等）。因此，控制省份固定效应，只能控制不随时间变化的因素，地理气候等等。
2. 可能存在“省份-时代”有关的未被观察的变量，同时影响改革的时间和儿童的outcome。（已经控制了“影响全国的时变因素”，和“时间不变的省际差异”，未控制影响各个省份自己的时变因素，如各省先后出台的政策等）

3. 用2个方法控制上述因素，


$$
S_{ipkt}^{M} = \gamma_0 + \gamma_1 Exposure_{pk}^{M} + \gamma_2 X_{it} + \theta_{k}^{M} + \tau_{p}^{M} + Trend_{pk}^{M} + \omega_{t} + \varepsilon_{ipkt}
$$

考虑CSLs政策如何影响母亲的教育程度（工具变量的一阶段回归）

其中：

1. $Exposure_{pk}^{M}$：队列k和省p的政策实施状态。注意这里是没有i的，即同一个省份同一年代的人，这个值相同。


把母亲分成3组：

1. 年龄在5-10岁为完全暴露组
2. 11-15岁为部分暴露组
3. 16岁及以上非暴露组


把exposure设置为0，1，2，其中0是无暴露（政策实施的时候，年龄已经太大了）2是完全暴露（政策实施，年龄较小）


In [37]:
reg_df[['mother_pid','mother_birth_y','mother_eduy','mother_birth_prov']]

,mother_pid,mother_birth_y,mother_eduy,mother_birth_prov
0,360160106.0,1979.0,9.0,NaN
1,370115104.0,1981.0,9.0,23.0
2,110011101.0,1973.0,6.0,34.0
3,110020105.0,1970.0,15.0,61.0
4,110022101.0,1978.0,15.0,42.0
...,...,...,...,...
10297,520211601.0,1988.0,NaN,NaN
10298,520213102.0,1964.0,NaN,52.0
10299,510390106.0,1991.0,NaN,51.0
10300,510401103.0,1979.0,0.0,51.0


In [38]:
# 计划生育政策实施的年份，和影响的第一代人
csl_df = pd.read_excel('./CSLs_year_prov.xlsx')
csl_df.head()

,Province,Time of initiation,Actual reform year,First affected birth cohort,prov_cn
0,Zhejiang,1985-09-01,1986,1971,浙江
1,Jiangxi,1986-02-01,1986,1971,江西
2,Beijing,1986-07-01,1986,1971,北京
3,Hebei,1986-07-01,1986,1971,河北
4,Shanxi,1986-07-01,1986,1971,山西


构造的逻辑：如果是当年9月1日之前颁布，那么就影响本届9月入学的同学。如果在9月1日及之后颁布，则影响下一年入学的同学。

In [39]:
1986-1971

15

### 构造Exposure

1. 浙江省，政策实际实施是1986年，将会影响的第一代人是1971年（1986-1971=15岁），这是部分暴露组中年龄最大的（受到影响的第一代人）
2. 因此，分组依据就是“实施年 - 出生年 = 政策实施时候的年龄”，然后按上述分组。71年=15岁，72年=14岁 ...
3. reg_df['mother_birth_prov']



In [40]:
data_str = '11:北京市, 12:天津市, 13:河北省, 14:山西省, 15:内蒙古自治区, 21:辽宁省, 22:吉林省, 23:黑龙江省, 31:上海市, 32:江苏省, 33:浙江省, 34:安徽省, 35:福建省, 36:江西省, 37:山东省, 41:河南省, 42:湖北省, 43:湖南省, 44:广东省, 45:广西, 46:海南省, 50:重庆市, 51:四川省, 52:贵州省, 53:云南省, 54:西藏自治区, 61:陕西省, 62:甘肃省, 63:青海省, 64:宁夏, 65:新疆, 99:数据缺失'
data_str = data_str.replace('省','').replace('市','').replace('自治区','')
# 将字符串分割为单独的省份数据
data_list = data_str.split(', ')

# 分割每个省份数据为id和名称
data_split = [item.split(':') for item in data_list]

# 创建DataFrame
prov_id_cn = pd.DataFrame(data_split, columns=['prov_id', 'prov_cn'])

prov_id_cn.head()

,prov_id,prov_cn
0,11,北京
1,12,天津
2,13,河北
3,14,山西
4,15,内蒙古


In [41]:
csl_df = safe_merge(csl_df, prov_id_cn)
csl_df.prov_id = csl_df.prov_id.astype(int)
csl_df.info()

Merge的方式是how='left'
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Province                     31 non-null     object        
 1   Time of initiation           31 non-null     datetime64[ns]
 2   Actual reform year           31 non-null     int64         
 3   First affected birth cohort  31 non-null     int64         
 4   prov_cn                      31 non-null     object        
 5   prov_id                      31 non-null     int32         
dtypes: datetime64[ns](1), int32(1), int64(2), object(2)
memory usage: 1.5+ KB


In [42]:
# 省份当然是有重复的
expos_df=safe_merge(reg_df[['mother_pid','wav','mother_birth_prov','mother_birth_y','mother_eduy']],csl_df,left_on='mother_birth_prov',right_on='prov_id')
expos_df.head()

Merge的方式是how='left'
注意：合并后的 DataFrame 有 10274/10302 个重复键值。第一个重复键值为: mother_birth_prov    11.0
Name: 7, dtype: float64


,mother_pid,wav,mother_birth_prov,mother_birth_y,mother_eduy,Province,Time of initiation,Actual reform year,First affected birth cohort,prov_cn,prov_id
0,360160106.0,2014,NaN,1979.0,9.0,NaN,NaT,NaN,NaN,NaN,NaN
1,370115104.0,2014,23.0,1981.0,9.0,Heilongjiang,1986-07-01,1986.0,1971.0,黑龙江,23.0
2,110011101.0,2014,34.0,1973.0,6.0,Anhui,1987-09-01,1988.0,1973.0,安徽,34.0
3,110020105.0,2014,61.0,1970.0,15.0,Shaanxi,1987-09-01,1988.0,1973.0,陕西,61.0
4,110022101.0,2014,42.0,1978.0,15.0,Hubei,1987-03-01,1987.0,1972.0,湖北,42.0


In [43]:
expos_df['reform_at_age'] = expos_df['Actual reform year'] - expos_df['mother_birth_y']
expos_df.head()

,mother_pid,wav,mother_birth_prov,mother_birth_y,mother_eduy,Province,Time of initiation,Actual reform year,First affected birth cohort,prov_cn,prov_id,reform_at_age
0,360160106.0,2014,NaN,1979.0,9.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,370115104.0,2014,23.0,1981.0,9.0,Heilongjiang,1986-07-01,1986.0,1971.0,黑龙江,23.0,5.0
2,110011101.0,2014,34.0,1973.0,6.0,Anhui,1987-09-01,1988.0,1973.0,安徽,34.0,15.0
3,110020105.0,2014,61.0,1970.0,15.0,Shaanxi,1987-09-01,1988.0,1973.0,陕西,61.0,18.0
4,110022101.0,2014,42.0,1978.0,15.0,Hubei,1987-03-01,1987.0,1972.0,湖北,42.0,9.0


检查一下：

1. 370115104号样本，出生于黑龙江省，1981年出生。黑龙江省的政策是1986年正式实施，此时她1986年-1981年 = 5岁，为完全影响组。
2. 110020105号样本，出生于陕西，1970年。陕西政策1988年实施，此时她1988 - 1970年 = 18岁，为没有影响组。

构造expos = 母亲出生的省份 -> 该省的改革年 -> 改革时母亲几岁 -> expos状态

In [44]:


def age_to_expos(age):
    """
    1. 年龄在5-10岁为完全暴露组 2
    2. 11-15岁为部分暴露组 1
    3. 16岁及以上非暴露组 0 

    把exposure设置为0，1，2，其中0是无暴露（政策实施的时候，年龄已经太大了）2是完全暴露（政策实施，年龄较小）
    """
    if age >= 16:
        return 0
    elif age >= 11:
        return 1
    else:
        return 2

expos_df['expos'] = expos_df['reform_at_age'].map(age_to_expos)
expos_df.head(10)

,mother_pid,wav,mother_birth_prov,mother_birth_y,mother_eduy,Province,Time of initiation,Actual reform year,First affected birth cohort,prov_cn,prov_id,reform_at_age,expos
0,360160106.0,2014,NaN,1979.0,9.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,2
1,370115104.0,2014,23.0,1981.0,9.0,Heilongjiang,1986-07-01,1986.0,1971.0,黑龙江,23.0,5.0,2
2,110011101.0,2014,34.0,1973.0,6.0,Anhui,1987-09-01,1988.0,1973.0,安徽,34.0,15.0,1
3,110020105.0,2014,61.0,1970.0,15.0,Shaanxi,1987-09-01,1988.0,1973.0,陕西,61.0,18.0,0
4,110022101.0,2014,42.0,1978.0,15.0,Hubei,1987-03-01,1987.0,1972.0,湖北,42.0,9.0,2
5,110041101.0,2014,11.0,1968.0,12.0,Beijing,1986-07-01,1986.0,1971.0,北京,11.0,18.0,0
6,110047102.0,2014,43.0,1977.0,12.0,Hunan,1991-09-01,1992.0,1977.0,湖南,43.0,15.0,1
7,110078104.0,2014,11.0,1974.0,12.0,Beijing,1986-07-01,1986.0,1971.0,北京,11.0,12.0,1
8,110079102.0,2014,13.0,1973.0,12.0,Hebei,1986-07-01,1986.0,1971.0,河北,13.0,13.0,1
9,110092104.0,2014,11.0,1963.0,12.0,Beijing,1986-07-01,1986.0,1971.0,北京,11.0,23.0,0


In [45]:
%%R -i expos_df
ls()

c:\Users\lee\anaconda3\Lib\site-packages\rpy2\robjects\pandas2ri.py:65: UserWarning: Error while trying to convert the column "Province". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '
c:\Users\lee\anaconda3\Lib\site-packages\rpy2\robjects\pandas2ri.py:65: UserWarning: Error while trying to convert the column "prov_cn". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '


[1] "check"             "expos_df"          "lm0"              
[4] "lm1"               "reg_df"            "screenreg_unicode"


In [46]:
%%R

lm0 = feols(mother_eduy ~ expos,data = expos_df)
check(list(lm0))


                       Model 1    
----------------------------------
(Intercept)               5.76 ***
                         (0.11)   
expos                     0.94 ***
                         (0.07)   
----------------------------------
Num. obs.              8518       
R^2 (full model)          0.02    
R^2 (proj model)                  
Adj. R^2 (full model)     0.02    
Adj. R^2 (proj model)             
*** p < 0.01; ** p < 0.05; * p < 0.1
 FALSE

NOTE: 1,784 observations removed because of NA values (LHS: 1,784).


可知：渐进实施的政策，确实影响了母亲的教育程度。

### 构造Trend

考虑$Trend_{pk}^{M}$，只有p和k

In [50]:
x = expos_df.query('Province == "Henan"')
x['cohort'] = x['mother_birth_y'] -  min(x['mother_birth_y']) + 1
x.head()

C:\Users\lee\AppData\Local\Temp\ipykernel_1636\3989135574.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['cohort'] =x['mother_birth_y'] -  min(x['mother_birth_y']) + 1


,mother_pid,wav,mother_birth_prov,mother_birth_y,mother_eduy,Province,Time of initiation,Actual reform year,First affected birth cohort,prov_cn,prov_id,reform_at_age,expos,cohort
189,410383104.0,2014,41.0,1984.0,7.0,Henan,1986-10-01,1987.0,1972.0,河南,41.0,3.0,2,47.0
219,410925101.0,2014,41.0,1981.0,0.0,Henan,1986-10-01,1987.0,1972.0,河南,41.0,6.0,2,44.0
220,410955104.0,2014,41.0,1976.0,9.0,Henan,1986-10-01,1987.0,1972.0,河南,41.0,11.0,1,39.0
221,410982105.0,2014,41.0,1981.0,6.0,Henan,1986-10-01,1987.0,1972.0,河南,41.0,6.0,2,44.0
367,411603105.0,2014,41.0,1975.0,0.0,Henan,1986-10-01,1987.0,1972.0,河南,41.0,12.0,1,38.0


In [53]:
result = []

for i,x in expos_df.groupby('prov_cn'):
    x['trend'] = x['mother_birth_y'] -  min(x['mother_birth_y']) + 1
    result.append(x)

expos_df = pd.concat(result)
expos_df

,mother_pid,wav,mother_birth_prov,mother_birth_y,mother_eduy,Province,Time of initiation,Actual reform year,First affected birth cohort,prov_cn,prov_id,reform_at_age,expos,trend
429,311977103.0,2014,31.0,1986.0,12.0,Shanghai,1986-09-01,1987.0,1972.0,上海,31.0,1.0,2,43.0
439,311936103.0,2014,31.0,1977.0,12.0,Shanghai,1986-09-01,1987.0,1972.0,上海,31.0,10.0,2,34.0
463,310942102.0,2014,31.0,1971.0,16.0,Shanghai,1986-09-01,1987.0,1972.0,上海,31.0,16.0,0,28.0
464,311058101.0,2014,31.0,1979.0,16.0,Shanghai,1986-09-01,1987.0,1972.0,上海,31.0,8.0,2,36.0
465,311093103.0,2014,31.0,1980.0,12.0,Shanghai,1986-09-01,1987.0,1972.0,上海,31.0,7.0,2,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9392,230180103.0,2020,23.0,1974.0,6.0,Heilongjiang,1986-07-01,1986.0,1971.0,黑龙江,23.0,12.0,1,12.0
9393,230421101.0,2020,23.0,1977.0,0.0,Heilongjiang,1986-07-01,1986.0,1971.0,黑龙江,23.0,9.0,2,15.0
9394,230465101.0,2020,23.0,1963.0,9.0,Heilongjiang,1986-07-01,1986.0,1971.0,黑龙江,23.0,23.0,0,1.0
9588,410637104.0,2020,23.0,1972.0,12.0,Heilongjiang,1986-07-01,1986.0,1971.0,黑龙江,23.0,14.0,1,10.0


In [64]:
reg_df2 = safe_merge(reg_df, expos_df[['mother_pid','wav','expos','trend','reform_at_age']]).query('reform_at_age>=5')
reg_df2.head()

Merge的方式是how='left'
注意：合并后的 DataFrame 有 3630/12524 个重复键值。第一个重复键值为: mother_pid    110011101.0
wav                2014.0
Name: 3, dtype: float64


,mother_pid,wav,child_pid,child_birth_y,child_age,child_gender,child_enroll,child_math,child_enroll2,child_eduy,...,mother_eth,mother_enroll2,mother_eduy,mother_wav,child_birth_prov,child_eth,mother_birth_prov,expos,trend,reform_at_age
1,370115104.0,2014,370115107.0,2004.0,10.0,1.0,1.0,1.0,1.0,3.0,...,NaN,0.0,9.0,2014.0,NaN,NaN,23.0,2.0,19.0,5.0
2,110011101.0,2014,110011103.0,1999.0,15.0,0.0,1.0,4.0,1.0,8.0,...,NaN,0.0,6.0,2014.0,NaN,NaN,34.0,1.0,7.0,15.0
3,110011101.0,2014,110011103.0,1999.0,15.0,0.0,1.0,4.0,1.0,8.0,...,NaN,0.0,6.0,2014.0,NaN,NaN,34.0,1.0,7.0,15.0
4,110020105.0,2014,110020107.0,2001.0,13.0,0.0,1.0,3.0,1.0,7.0,...,NaN,0.0,15.0,2014.0,NaN,NaN,61.0,0.0,5.0,18.0
5,110022101.0,2014,110022103.0,2001.0,13.0,0.0,1.0,2.0,1.0,5.0,...,NaN,0.0,15.0,2014.0,NaN,NaN,42.0,2.0,11.0,9.0


### 有Trend的回归


In [65]:
%%R -i reg_df2
ls()

[1] "check"             "expos_df"          "lm0"              
[4] "lm1"               "reg_df"            "reg_df2"          
[7] "screenreg_unicode"


In [68]:
%%R

lm0 = feols(mother_eduy ~ expos,data = reg_df2)
lm1 = feols(mother_eduy ~ expos + child_age + child_gender | wav + mother_birth_y + mother_birth_prov, data=reg_df2 )
check(list(lm0,lm1))


                                Model 1      Model 2   
-------------------------------------------------------
(Intercept)                        5.46 ***            
                                  (0.11)               
expos                              1.10 ***     0.71 **
                                  (0.07)       (0.12)  
child_age                                       0.01   
                                               (0.01)  
child_gender                                    0.41 **
                                               (0.09)  
-------------------------------------------------------
Num. obs.                       5490         5490      
R^2 (full model)                   0.04         0.30   
R^2 (proj model)                                0.00   
Adj. R^2 (full model)              0.04         0.29   
Adj. R^2 (proj model)                           0.00   
Num. groups: wav                                3      
Num. groups: mother_birth_y                    

NOTE: 125 observations removed because of NA values (LHS: 125).
NOTE: 125 observations removed because of NA values (LHS: 125).


In [88]:
%%R
lm0 = feols(child_math ~ mother_eduy, data = reg_df2)
lm1 = feols(child_math ~ mother_eduy + child_age + child_gender | wav + mother_birth_y + mother_birth_prov, data = reg_df2)
iv_s1 = feols(mother_eduy ~ expos + child_age + child_gender | wav + mother_birth_y + mother_birth_prov, data = reg_df2)

reg_df2['mother_eduy_fitted'] =  fitted(iv_s1)

   [1]  9.557295e+00  5.955418e+00  5.955418e+00  5.893028e+00  8.794353e+00
   [6]  9.784424e+00  9.923150e+00  1.036877e+01  1.036877e+01  7.006780e+00
  [11]  9.042846e+00  7.975597e+00  9.983947e+00  9.745708e+00  9.745708e+00
  [16]  9.567931e+00  1.090063e+01  1.161679e+01  1.113177e+01  1.134789e+01
  [21]  1.164870e+01  8.709580e+00  1.030605e+01  1.080668e+01  1.093253e+01
  [26]  1.167203e+01  1.127158e+01  9.271982e+00  8.992443e+00  8.193772e+00
  [31]  7.799131e+00  7.799131e+00  8.003105e+00  9.694506e+00  8.966902e+00


  [36]  8.945630e+00  8.361147e+00  8.860895e+00  9.382258e+00  7.981832e+00
  [41]  7.965690e+00  7.417867e+00  9.793652e+00  9.783016e+00  8.744418e+00
  [46]  7.006780e+00  8.945630e+00  8.914740e+00  8.361147e+00  8.982752e+00
  [51]  7.417867e+00  8.072870e+00  7.006780e+00  7.417867e+00  8.740325e+00
  [56]  8.534543e+00  1.017283e+01  8.956266e+00  8.398049e+00  8.992443e+00
  [61]  9.392894e+00  8.828987e+00  8.761597e+00  8.956266e+00  8.729689e+00
  [66]  7.017416e+00  7.017416e+00  7.549473e+00  8.740325e+00  8.729689e+00
  [71]  8.003105e+00  7.960560e+00  7.592018e+00  9.392894e+00  7.428503e+00
  [76]  7.019366e+00  7.019366e+00  6.294142e+00  8.971171e+00  9.392894e+00
  [81]  8.765690e+00  8.003105e+00  8.545179e+00  7.592018e+00  7.592018e+00
  [86]  9.187413e+00  6.429134e+00  8.945630e+00  9.793652e+00  9.793652e+00
  [91]  7.981832e+00  7.618284e+00  1.020474e+01  9.793652e+00  9.793652e+00
  [96]  9.772380e+00  7.450922e+00  9.382258e+00  8.755054e+00  8.755054e+00

Exception ignored from cffi callback <function _consolewrite_ex at 0x000002818007E200>:
Traceback (most recent call last):
  File "c:\Users\lee\anaconda3\Lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 136, in _consolewrite_ex
    consolewrite_print(s)
  File "c:\Users\lee\anaconda3\Lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 119, in consolewrite_print
    print(s, end='', flush=True)
  File "c:\Users\lee\anaconda3\Lib\site-packages\ipykernel\iostream.py", line 564, in flush
    if not evt.wait(self.flush_timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lee\anaconda3\Lib\threading.py", line 622, in wait
    signaled = self._cond.wait(timeout)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lee\anaconda3\Lib\threading.py", line 324, in wait
    gotit = waiter.acquire(True, timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 



[3821]  7.295703e+00  8.516906e+00  5.256215e+00  4.877036e+00  6.294249e+00
[3826]  7.685518e+00  6.863208e+00  6.863208e+00  1.016232e+01  8.497987e+00
[3831]  6.837812e+00  8.850898e+00  1.058202e+01  8.679044e+00  9.105897e+00
[3836]  9.527621e+00  1.040155e+01  9.111403e+00  8.297569e+00  9.624164e+00
[3841]  9.624164e+00  9.580520e+00  7.363103e+00  6.708204e+00  8.257239e+00
[3846]  8.257239e+00  8.551683e+00  6.760446e+00  6.760446e+00  8.214694e+00
[3851]  8.185197e+00  8.261508e+00  8.261508e+00  6.871718e+00  6.871718e+00
[3856]  7.688386e+00  8.331696e+00  8.331696e+00  8.585647e+00  8.585647e+00
[3861]  8.519774e+00  8.024118e+00  6.802991e+00  6.802991e+00  6.676295e+00
[3866]  7.619575e+00  1.014329e+01  8.163924e+00  6.781718e+00  6.781718e+00
[3871]  6.781718e+00  6.861082e+00  6.243936e+00  6.254572e+00  1.075545e+01
[3876]  1.075545e+01  8.045391e+00  5.630420e+00  7.266527e+00  8.530411e+00
[3881]  8.530411e+00  8.062570e+00  8.062570e+00  6.719734e+00  6.665659e+0

NOTE: 281 observations removed because of NA values (LHS: 165, RHS: 125).
NOTE: 281 observations removed because of NA values (LHS: 165, RHS: 125).
NOTE: 125 observations removed because of NA values (LHS: 125).


In [ ]:

iv_s2 = feols(child_math ~ mother_eduy_fitted + + child_age + child_gender | wav + mother_birth_y + mother_birth_prov, data = reg_df2)
# iv1 = feols(child_math ~ mother_eduy  + child_age + child_gender | wav + mother_birth_y + mother_birth_prov | expos ~  child_age + child_gender, data=reg_df2 )

check(list(lm0,lm1,iv_s1),custom.model.names = c("ols","ols",'iv_s1'))

In [77]:
%%R
check(list(iv1))


                                Model 1 
----------------------------------------
fit_expos                         53.86 
                                (952.52)
mother_eduy                       -0.20 
                                  (2.76)
child_age                          0.33 
                                  (4.08)
----------------------------------------
Num. obs.                       5334    
Num. groups: wav                   3    
Num. groups: mother_birth_y       37    
Num. groups: mother_birth_prov    27    
R^2 (full model)                -157.64 
R^2 (proj model)                -165.41 
Adj. R^2 (full model)           -159.66 
Adj. R^2 (proj model)           -165.51 
*** p < 0.01; ** p < 0.05; * p < 0.1
 FALSE